In [ ]:
import redisai
import ml2rt
from PIL import Image
from torchvision import transforms
import numpy as np

class Object(object): pass

In [ ]:
tr = Object()
tr.redis = redisai.Client()

In [ ]:
# docker run -p 6379:6379 -it --rm redisai/redisai

In [ ]:
tr.loaded_model = ml2rt.load_model('plant.pt')

In [ ]:
tr.redis.modelset(
    "plant", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

In [ ]:
!ls ../../input/plant-pathology-2020-fgvc7/test/rust

In [ ]:
input_path = "../../input/plant-pathology-2020-fgvc7"
tr.img = Image \
    .open(input_path + '/test/rust/Train_109.jpg') \
    .convert("RGB")
tr.img

In [ ]:
tr.transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, .224, .225])
])

In [ ]:
tr.img_t = tr.transform(tr.img)

In [ ]:
tr.img_t.shape

In [ ]:
tr.img_t = tr.img_t.numpy()

In [ ]:
tr.img_t = np.expand_dims(tr.img_t, axis=0)
tr.img_t.shape

In [ ]:
tr.redis.tensorset('input', tr.img_t)

In [ ]:
tr.redis.modelrun('plant', 'input', 'pred')

In [ ]:
pred = tr.redis.tensorget('pred')

In [ ]:
pred

In [ ]:
def softmax(v):
    return np.exp(v) / np.sum(np.exp(v))

In [ ]:
softmax(pred)

Our labels are:
```
['healthy', 'multiple_diseases', 'rust', 'scab']
```

### Twitter Loop

In [ ]:
import io, urllib.request, time, twitter, os, PIL

def twitter_loop():
    msg = Object()
    msg.running = True
    
    api = twitter.Api(
        consumer_key=os.environ['TWITTER_CONSUMER_KEY'],
        consumer_secret=os.environ['TWITTER_CONSUMER_SECRET'],
        access_token_key=os.environ['TWITTER_TOKEN_KEY'],
        access_token_secret=os.environ['TWITTER_TOKEN_SECRET'])   
    
    labels = ['healthy', 'multiple_diseases', 'rust', 'scab']
    
    timeline = api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True)    
    
    since_id = since_id = timeline[0].id \
        if len(timeline) > 0 \
        else 0

    while msg.running:
        msg.t = api.GetUserTimeline(screen_name="alexonsoftware", 
                            count=10, 
                            include_rts=False,
                            exclude_replies=True,
                            since_id=since_id)
        if len(msg.t) > 0:
            for msg.tweet in msg.t:
                if 'media' in msg.tweet.AsDict():
                    msg.media_list = msg.tweet.AsDict()['media']
                    for msg.pic in msg.media_list:
                        msg.img = PIL.Image.open(
                            io.BytesIO(
                                urllib.request.urlopen(
                                    msg.pic['media_url']).read())).convert('RGB')
                        msg.img_t = tr.transform(msg.img).unsqueeze(0)
                        
                        tr.redis.tensorset('image', msg.img_t.numpy())
                        
                        tr.redis.modelrun('plant', 'image', 'pred')

                        msg.pred = tr.redis.tensorget('pred')[0]
                        
                        msg.pred = softmax(msg.pred)
                        print(msg.pred)
                        msg.ind = range(len(labels))
                        print(msg.ind)
                        result = ''
                        for msg.p in msg.ind:
                            result += f"{labels[msg.p]} : {msg.pred[msg.p]}\n"
                        print(result)
                        api.PostUpdate(
                            status=result,
                            in_reply_to_status_id=msg.tweet.id
                        )
                since_id = max(since_id, msg.tweet.id)
        time.sleep(1)
        msg.running = True

In [ ]:
twitter_loop()